In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import logging

# Konfiguration des Logging-Systems
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

BASE_URL = "https://www.eversports.de"

def scrape_workshops(url):
    """
    Scraped Workshop-Informationen von einer gegebenen URL.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
    except requests.RequestException as e:
        logging.error(f"Fehler beim Abrufen der Webseite {url}: {e}")
        return None

    soup = BeautifulSoup(response.content, 'html.parser')
    workshop_data = {
        'Workshop Name': [],
        'Workshop Date': [],
        'Workshop Price': [],
        'Workshop Units': [],
        'Workshop Studio': [],
        'Workshop Address': [],
        'Workshop Href': []
    }

    workshop_elements = soup.find_all('a', class_='marketplace-tile js_marketplace-tile')
    for workshop in workshop_elements:
        workshop_data['Workshop Name'].append(workshop.find('h4').text)
        workshop_data['Workshop Date'].append(workshop.find('div', class_='marketplace-tile__date').text)
        workshop_data['Workshop Price'].append(workshop.find('div', class_='marketplace-tile__price').text)
        workshop_data['Workshop Units'].append(workshop.find('small', class_='u-text-bold').text)
        
        workshop_content = workshop.find('div', class_='marketplace-tile__content__bottom').find_all('small')
        workshop_data['Workshop Studio'].append(workshop_content[0].text)
        workshop_data['Workshop Address'].append(workshop_content[1].text)
        
        workshop_data['Workshop Href'].append(BASE_URL + workshop['href'])

    workshoplist_df = pd.DataFrame(workshop_data)
    workshoplist_df.drop(columns=['Workshop Href'], inplace=True)

    return workshoplist_df

if __name__ == "__main__":
    test_url = "https://www.eversports.de/sw/poda-studio"
    df = scrape_workshops(test_url)
    if df is not None:
        logging.info(f"Gescrapte Workshop-Daten:\n{df}")
    else:
        logging.info("Keine Daten gescraped.")


2023-12-23 15:49:22,766 - INFO - Gescrapte Workshop-Daten:
                              Workshop Name  Workshop Date Workshop Price  \
0                     Aerial Loops Workshop   Jan13Samstag     ab 35,00 €   
1               Handstands on Pole Workshop   Jan20Samstag     ab 35,00 €   
2                    How to invert Workshop   Jan20Samstag     ab 35,00 €   
3  Rock Hard Style High Heels Pole Workshop   Feb04Sonntag     ab 35,00 €   
4              Hip Hop Pole Choreo Workshop  März16Samstag     ab 35,00 €   

  Workshop Units Workshop Studio                           Workshop Address  
0      1 Einheit     poda Studio  10-12 Martinstraße Innenhof, 52062 Aachen  
1      1 Einheit     poda Studio  10-12 Martinstraße Innenhof, 52062 Aachen  
2      1 Einheit     poda Studio  10-12 Martinstraße Innenhof, 52062 Aachen  
3      1 Einheit     poda Studio  10-12 Martinstraße Innenhof, 52062 Aachen  
4      1 Einheit     poda Studio  10-12 Martinstraße Innenhof, 52062 Aachen  
